In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow as tf

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, cross_val_score
from tensorflow.keras import Sequential, Input, layers
from tensorflow_addons.metrics import FBetaScore
from tensorflow.keras.metrics import Recall, Precision


import sys
sys.path.append('../src')

from src.utils.data import get_csv_from_bq,select_tomo_ids

#from utils.model_plot import plot_history2
from src.utils.model_plot import plot_history_regression
from src.ml_logic.preprocess import batches_images_ram,selection_images_labels,read_img_jpg

2025-06-05 15:59:51.365035: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import os
import glob
import sys
sys.path.append('../src')

from src.utils.data import get_csv_from_bq,select_tomo_ids

def selection_images_labels(df, num_slices=[300,500,800], num_motors=[0,1]):

   # Step 1: Filter tomos
    tomo_ids = 'tomo_0a8f05', 'tomo_0a180f', 'tomo_0c3a99', 'tomo_0c3d78'
    #select_tomo_ids(df, number_of_slices=num_slices, number_of_motors=num_motors)
    df_select = df[df['tomo_id'].isin(tomo_ids)].copy()

    # Step 2: Set up labels
   # df_select['Number_of_motors'] = df_select.apply(lambda row: (row['Motor_axis_2'], row['Motor_axis_1']), axis=1)

    # Step 3: Load all images
    dir_mean_image = '../data/pictures_process/train_process/'
    all_images = glob.glob(os.path.join(dir_mean_image, '**', '*.jpg'), recursive=True)

    print(f"Found {len(all_images)} images in {dir_mean_image}")

    # Step 4: Match images using substring matching
    filtered_image_paths = []
    labels = []

    for _, row in df_select.iterrows():
        tomo_id = row['tomo_id']
        print(tomo_id)
        matched = [print(p) for p in all_images if tomo_id in os.path.basename(p)]

        if matched:
            filtered_image_paths.append(matched[0])  # If multiple, take the first
            labels.append(row['Number_of_motors'])
        else:
            print(f"⚠️ No image found for tomo_id: {tomo_id}")

    print(f"Matched {len(filtered_image_paths)} image-label pairs")

    labels = np.array(labels, dtype=np.float32)
    return filtered_image_paths, labels


# Define image reading function
def read_img_jpg(path, label):

    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.cast(img, tf.float32) / 255.0  # normalize to [0, 1]
    return img, label



def batches_images_ram(
    read_img_jpg,
    filtered_image_paths,
    labels,
    shuffle=True,
    batch_size=32,
    split=False,
    val_fraction=0.2,
    test_fraction=0.2,
    seed=42):


    dataset_size = len(filtered_image_paths)
    # Combine and optionally shuffle the data as a list of tuples
    data = list(zip(filtered_image_paths, labels))
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(data)

    # Unzip the shuffled data back
    filtered_image_paths, labels = zip(*data)

    # Convert back to lists or arrays
    filtered_image_paths = list(filtered_image_paths)
    labels = list(labels)

    if split:
        # Compute sizes
        val_size = int(val_fraction * dataset_size)
        test_size = int(test_fraction * dataset_size)
        train_size = dataset_size - val_size - test_size

        # Split into slices
        test_paths = filtered_image_paths[:test_size]
        test_labels = labels[:test_size]

        val_paths = filtered_image_paths[test_size:test_size + val_size]
        val_labels = labels[test_size:test_size + val_size]

        train_paths = filtered_image_paths[test_size + val_size:]
        train_labels = labels[test_size + val_size:]

        # Create tf.data.Dataset for each
        train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels)).map(read_img_jpg).batch(batch_size)
        val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels)).map(read_img_jpg).batch(batch_size)
        test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels)).map(read_img_jpg).batch(batch_size)

        return train_ds, val_ds, test_ds, test_paths, test_labels

    else:
        # Single dataset
        dataset = tf.data.Dataset.from_tensor_slices((filtered_image_paths, labels))
        dataset = dataset.map(read_img_jpg).batch(batch_size)
        return dataset, filtered_image_paths, labels


In [45]:
selection_images_labels(df, num_slices=[300,500,800], num_motors=[0,1])

Found 3200 images in ../data/pictures_process/train_process/
tomo_0a180f
⚠️ No image found for tomo_id: tomo_0a180f
tomo_0c3d78
⚠️ No image found for tomo_id: tomo_0c3d78
tomo_0a8f05
⚠️ No image found for tomo_id: tomo_0a8f05
tomo_0c3a99
⚠️ No image found for tomo_id: tomo_0c3a99
Matched 0 image-label pairs


([], array([], dtype=float32))

In [25]:
import os
import re
import numpy as np
from PIL import Image
from src.utils.image_padding import padd_picture

def pad_stack_z_with_2d_padding(
    folder_path,
    padded_folder_path,
    xy_size,
    target_depth=800,
    pad_value=255
):
    """
    Pads all slices in a folder to (xy_size, xy_size) and pads stack to target_depth.
    Saves padded 2D images in padded_folder_path.

    Returns:
    --------
    np.ndarray: padded stack of shape (target_depth, xy_size, xy_size, 3)
    """

    def numeric_sort_key(filename):
        numbers = re.findall(r'\d+', filename)
        return int(numbers[0]) if numbers else -1

    # Get sorted image paths with numeric sorting
    image_files = sorted(
        [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))],
        key=numeric_sort_key
    )

    padded_images = []
    for file in image_files:
        source_path = os.path.join(folder_path, file)
        dest_path = os.path.join(padded_folder_path, file)

        # Pad 2D image to xy_size x xy_size and save
        padd_picture(source_path, dest_path, xy_size)

        # Load padded image as numpy array
        img = Image.open(dest_path).convert('RGB')
        padded_images.append(np.array(img))

    stack = np.stack(padded_images, axis=0)  # (Z, xy_size, xy_size, 3)
    current_depth = stack.shape[0]
    print(current_depth)
    if current_depth > target_depth:
        raise ValueError(f"Stack depth {current_depth} exceeds target {target_depth}")

    pad_slices = target_depth - current_depth
    print(pad_slices)
    if pad_slices > 0:
        # Save white padding slices as images on disk
        for i in range(pad_slices):
            pad_slice_index = current_depth + i + 1
            pad_filename = f"slice_{pad_slice_index}.jpg"  # or adapt naming
            pad_filepath = os.path.join(padded_folder_path, pad_filename)

            white_img = Image.new('RGB', (xy_size, xy_size), (pad_value, pad_value, pad_value))
            white_img.save(pad_filepath)

            # Append white slice to the stack list
            padded_images.append(np.array(white_img))

        # Recreate stack with padding slices included
        stack = np.stack(padded_images, axis=0)

    return stack

In [30]:
df[df['tomo_id']=='tomo_0a8f05']
df[df['tomo_id']=='tomo_0a180f']
df[df['tomo_id']=='tomo_0c3a99']
df[df['tomo_id']=='tomo_0c3d78']


,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
50,50,35,tomo_0c3d78,-1.0,-1.0,-1.0,800,928,960,13.1,0


In [31]:
padded_stack = pad_stack_z_with_2d_padding(
    folder_path='../data/pictures_raw/train/tomo_0c3d78',
    padded_folder_path='../data/pictures_process/train_process/tomo_0c3d78_padded',
    xy_size=960,
    target_depth=800,
    pad_value=255  # white background
)

print(padded_stack.shape)  # (800, 960, 960, 3)


800
0
(800, 960, 960, 3)


In [32]:
path_train_csv= '../data/csv_raw/train_labels.csv'
path_image= '../data/pictures_process/train_process'

df = pd.read_csv(path_train_csv).copy()

In [33]:
df = pd.read_csv('../data/csv_raw/train_labels.csv')
df.head(10)

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
0,0,16,tomo_049310,-1.0,-1.0,-1.0,500,924,956,19.7,0
1,1,30,tomo_098751,-1.0,-1.0,-1.0,500,924,956,16.1,0
2,2,54,tomo_136c8d,-1.0,-1.0,-1.0,500,924,956,19.7,0
3,3,59,tomo_146de2,-1.0,-1.0,-1.0,500,924,956,16.1,0
4,4,84,tomo_1dc5f9,-1.0,-1.0,-1.0,500,924,956,19.7,0
5,5,121,tomo_28f9c1,-1.0,-1.0,-1.0,500,924,956,16.1,0
6,6,173,tomo_39b15b,-1.0,-1.0,-1.0,500,924,956,16.1,0
7,7,181,tomo_3b8291,-1.0,-1.0,-1.0,500,924,956,19.7,0
8,8,193,tomo_40b215,-1.0,-1.0,-1.0,500,924,956,16.1,0
9,9,219,tomo_4baff0,-1.0,-1.0,-1.0,500,924,956,16.1,0


In [36]:
df_select = df[df['tomo_id'].isin(['tomo_0a8f05', 'tomo_0a180f', 'tomo_0c3a99', 'tomo_0c3d78'])].copy()
df_select

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
49,49,31,tomo_0a180f,-1.0,-1.0,-1.0,800,928,960,13.1,0
50,50,35,tomo_0c3d78,-1.0,-1.0,-1.0,800,928,960,13.1,0
292,292,32,tomo_0a8f05,52.0,587.0,575.0,300,928,928,13.1,1
343,343,34,tomo_0c3a99,125.0,645.0,227.0,300,959,928,15.6,1


In [ ]:
def initiate_model3D():

    model = Sequential()
     ### First Convolution & MaxPooling
    model.add(Input(shape=(X.shape)))
    model.add(layers.Conv3D(32, (3,3,3), activation = 'relu',padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(3, 3, 3)))
    model.add(layers.Conv3D(64, (4,4,4), activation = 'relu',padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(3, 3, 3)))

    ## Flattening
    model.add(layers.Flatten())

    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(10,activation='relu'))

    model.add(layers.Dropout(0.3))  # Optional
    model.add(layers.Dense(1,activation='sigmoid'))

    return model

In [15]:
model = initiate_model3D()
model.summary()


ValueError: Input 0 of layer "conv3d" is incompatible with the layer: expected min_ndim=5, found ndim=4. Full shape received: (None, 960, 960, 1)

In [ ]:
import tensorflow as tf
def euclidean_loss(y_true, y_pred):
    return tf.reduce_mean(tf.norm(y_pred - y_true, axis=1))
def compile_model3D_regress(model):
    ### Model compilation
    #model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['FBetaScore',Recall(),'precision','accuracy'], run_eagerly=True)

    model.compile(loss= 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics=['accuracy',
                           FBetaScore(num_classes=1,
                                      average='micro',
                                      beta=2.0),
                           Recall(),
                        #    'precision',
                           ],)

    return model


In [12]:
from src.ml_logic.interface import train
model = initiate_model3D()
model = compile_model3D_regress(model)

In [ ]:
def model_train (model, X_train_processed, y_train, batch_size=16 ,epochs=100):
    verbose=1
    validation_split=0.1
    es = EarlyStopping(patience=3,
                       restore_best_weights=True)

    history = model.fit(X_train_processed,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        # validation_split=validation_split,
                        validation_data=(X_val, y_val),
                        callbacks=[es],
                        verbose=verbose)
    return history


NameError: name 'train_ds' is not defined